In [4]:
import pandas as pd
import numpy as np

In [10]:
"""
src: http://groups.mrl.uiuc.edu/chiang/czoschke/diffraction-selection-rules.html


Allowed planes
primitive - any
Body Centered - h+k+l = 2n even
face centered - h,k,l all odd or all even
fcc diamond - h,k,l all odd || h+k+l = 4n
HCP - L even, H+2K != 3n
"""
def getAllPlanes(n):
    df = pd.DataFrame(columns=["h", "k", "l"])
    for h in range(-n, n+1): 
        for k in range(-n, n+1): 
            for l in range(-n, n+1): 
                i = len(df)
                df.loc[i, "h"] = h
                df.loc[i, "k"] = k
                df.loc[i, "l"] = l
    return df

def isBodyCentered(x): 
    buff = sum(x)
    if buff % 2 == 0: 
        return True
    else:
        return False

def isFaceCentered(x):
    h, k, l = x
    if (h%2 == 1) and (k%2 == 1) and (l%2 == 1): 
        return True
    else:
        return False
    
def isFCCDiamond(x): 
    if isFaceCentered(x):
        return True
    if sum(x) % 8 == 1: 
        return True
    return False

def isHCP(x):
    h, k, l  = x
    if l%2 == 0: 
        return True
    buff = h  + 2 * k 
    if buff % 3 == 1: 
        return True
    return False

class dspace:
    def hcp(x, a, c): 
        h, k, l = x
        return 4/3 * (h ** 2 + h * k + k ** 2 ) / a ** 2 + l ** 2 / c ** 2
    def fcc(x, a): 
        h, k, l = x
        return (h **2 + k **2 + l ** 2 ) / a **2
    def tetragonal(x, a, c): 
        h, k, l = x
        return (h ** 2 + k ** 2) / a ** 2 + l ** 2 /  c** 2

In [134]:
"""
For Ba2Y(CuO2)3 
"""
df = getAllPlanes(5)
# df = df[df[["h", "k", "l"]].apply(isBodyCentered, axis=1)]
a = .3891 # nm
c = .12123
l = .1542 # nm Cu K alpha radiation
df["1/d2"] = df[["h", "k", "l"]].apply(lambda x: dspace.tetragonal(x, a, c), axis=1)
df["d"] = 1/ df["1/d2"]  ** .5
df["sinq"] = l / (2 *df["d"])
df["q_rad"] = np.arcsin(df["sinq"])
df["q_deg"] =  df["q_rad"] * 360/ 2/ np.pi
df["2q"] = df["q_deg"] * 2
df["sinq/l(A)"] = df["sinq"] / (l * 10)

/home/rose/.local/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in arcsin
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [135]:
"""
Scattering Factor of Each eleemnt
""" 
def scattering_factor(s, z, abPairs):
    """
    s = sinq /l (A)
    z = Z
    ab Pairs: List of tuples 
    """
    tot = 41.78214 * s ** 2 
    summation = 0
    for pair in abPairs:
        a, b = pair
        summation += a * np.exp(-b * s **2)
    return z - tot * summation



# for Ba
z = 56
abPairs = [(7.821, 117.657), (6.004, 18.778), (3.280, 3.263), (1.103, .376)]
df["f_Ba"] = df[["sinq/l(A)"]].apply(lambda x: scattering_factor(x, z, abPairs))
# for Y
z = 39
abPairs = [(4.129, 27.548), (3.012, 5.088), (1.179, .591)]
df["f_Y"] = df[["sinq/l(A)"]].apply(lambda x: scattering_factor(x, z, abPairs))
# for Cu
z =29
abPairs = [(1.579, 62.940), (1.820, 12.453), (1.658, 2.504), (.532, .333)]
df["f_Cu"] = df[["sinq/l(A)"]].apply(lambda x: scattering_factor(x, z, abPairs))
# for O
z = 8
abPairs = [(.455, 23.780), (.917, 7.622), (.472, 2.144), (.138, .296)]
df["f_O"] = df[["sinq/l(A)"]].apply(lambda x: scattering_factor(x, z, abPairs))


In [136]:
data = {
"Ba": [(1/2, 1/2, 0.19711400), (1/2,1/2,0.80288600)],
"Y": [(1/2, 1/2,  1/2)],
"Cu": [(0, 0,0.36812500),(0, 0, 0.63187500 ),(0, 0, 0)],
"O": [(0, 1/2, 0.62090900) , (1/2, 0,0.37909100), (1/2, 0, 0.62090900), (0, 1/2, 0.37909100), (0 , 0, 0.14934000), (0 , 0,0.85066000)]
}

In [137]:
def structureFactor(x, data): 
    def expTerm(coords, h, k, l): 
        tot = 0
        for coord in coords: 
            a,b,c = coord
            tot += np.exp(2 * np.pi * 1j * (a *h + b * k + c * l))
        tot = complex(tot)
        if abs(tot.imag) <= 1e-10: 
            tot = tot.real
        if abs(tot.real) <= 1e-10: 
            tot = tot.imag
        return tot
    
    
    h, k, l, f_ba, f_y, f_cu, f_o = x 

    baBuff = f_ba * expTerm(data["Ba"], h, k, l) 
    yBuff = f_y * expTerm(data["Y"], h, k, l)
    cuBuff = f_cu * expTerm(data["Cu"], h, k, l)
    oBuff = f_o * expTerm(data["O"], h, k, l)
    return baBuff + yBuff + cuBuff + oBuff
    
    
    
    
    
    

In [138]:
# If computations cant be made its a forbidden reflection
df = df.replace(np.inf, np.nan)
df = df.dropna()
# Stucture Factors
df["F"] = df[["h", "k", "l", "f_Ba", "f_Y", "f_Cu", "f_O"]].apply(lambda x: structureFactor(x, data), axis=1) 
df["F2"] = df["F"] ** 2

In [139]:
# Getting counts of each family
df["p"] = df.groupby("d")["d"].transform("count")

In [140]:
df["(1+cos2(2theta))/(sin2(theta))cos(theta))"]  = (1 + np.cos(df["q_rad"] ** 2) ** 2) / (np.sin(df["q_rad"]) ** 2 * np.cos(df["q_rad"]))
df["Intensity"] = df["F2"] * df["p"] * df["(1+cos2(2theta))/(sin2(theta))cos(theta))"]

In [141]:
df = df.sort_values("d")
df.to_csv("Ba2Y(CuO2).csv", index=False)


# Remove Duplicates
df["c"] = df[["h", "k", "l"]].apply(lambda x: x[0] ** 2 + x[1] **2 + x[2] ** 2 , axis =1 )
df["s"] =  df[["h", "k", "l"]].apply(lambda x: x[0]  + x[1] + x[2] , axis =1 )
df = df.sort_values("s",ascending = False)
df = df.drop_duplicates("c", keep="first")
df = df.drop(columns=["s", "c"])

df.to_csv("Ba2Y(CuO2)_Clean.csv", index=False)

In [142]:
df = pd.read_csv("nyloncoords.csv", names=range(12))

In [161]:
buff = {}
l = []
for i in  range(8): 
    d = df[range(9, 12)]
    if i < 6: 
        l.append(tuple(d.loc[i, :].to_list()))
        buff["C"] = l
    elif i == 6: 
        buff["N"] = tuple(d.loc[i, :].to_list())
    else : 
        buff["O"] = tuple(d.loc[i, :].to_list())